In [1]:
from noworkflow.now.persistence.models import Evaluation, Activation
from noworkflow.now.models.dependency_querier import DependencyQuerier
from noworkflow.now.models.dependency_querier.node_context import NodeContext
from noworkflow.now.models.dependency_querier.querier_options import QuerierOptions
from noworkflow.now.persistence.models import Trial
from copy import copy

import numpy as np
from pprint import pprint

In [2]:
import numpy as np

# Helper functions
def get_activation(activation_id, trial_id = None):
    if not trial_id:
        trial_id =  __noworkflow__.trial_id
    return Activation((trial_id, activation_id))

def get_evaluation(evaluation_id, trial_id = None):
    if not trial_id:
        trial_id =  __noworkflow__.trial_id
    return Evaluation((trial_id, evaluation_id))

def get_act_evals(activation_id):
    activation = get_activation(activation_id)
    return activation.evaluations    

def print_act_forward(activation_id=1):
    activation = get_activation(activation_id)
    
    for i in list(activation.dependent_dependencies):
        pprint.pprint(i.to_dict())
        
def print_act_backward(activation_id=1):
    activation = get_activation(activation_id)
    
    for i in list(activation.dependency_dependencies):
        pprint.pprint(i.to_dict())

def print_act_evals(activation_id=1):
    activation = get_activation(activation_id)

    for item in activation.evaluations:
        pprint.pprint(item.to_dict())
        


In [131]:
class NotebookQuerierOptions(QuerierOptions):
    def visit_arrow(self, context, neighbor):
        indent = 1
       
        global non_go

        if neighbor.evaluation.code_component.type == 'function_def':
            non_go = neighbor.evaluation.code_component.id
       
    
        if neighbor.arrow not in ('argument', '<.__class__>', 'item'):
            if context.evaluation.code_component.type not in ('add', 'literal', 'mult', 'div', 'param', 'sub', 'attribute', 'usub'):
                if neighbor.evaluation.code_component.type not in ('add', 'literal', 'mult', 'div', 'param', 'sub', 'attribute', 'usub'):
                    if (context.evaluation.code_component.type != 'call') and (neighbor.evaluation.code_component.type != 'call'):
                        if (context.evaluation.code_component.container_id != 'None'):
                            if neighbor.evaluation.code_component.container_id != non_go:
                                #print(context.evaluation, context.evaluation.code_component)
                                #print(neighbor.arrow, '->' , neighbor.evaluation, neighbor.evaluation.code_component)
                                code_comp = neighbor.evaluation.code_component
                                evaluation = context.evaluation
                                print(" container id = "+ str(code_comp.container_id) + "; variable = " + code_comp.name + ' <- ' + str(evaluation.repr))
                                print('_____')


In [132]:
querier = DependencyQuerier(options=NotebookQuerierOptions())
nodes_to_visit, visited, found = querier.navigate_dependencies([evals])

 container id = 706; variable = x <- 110246.0
_____
 container id = 706; variable = var_e <- 110236.0
_____
 container id = 645; variable = var_b <- -100
_____
 container id = 691; variable = var_c <- 110136.0
_____
 container id = 691; variable = var_d <- 330408
_____
 container id = 676; variable = var_c <- 1412
_____
 container id = 652; variable = y <- 30
_____
 container id = 652; variable = x <- 50
_____
 container id = 479; variable = stress_func <- <function stress_func at 0x7fef95a4c5f0>
_____
 container id = None; variable = int <- <class 'int'>
_____
 container id = 479; variable = mae <- <function mae at 0x7fef95a4c050>
_____
 container id = 479; variable = rmse <- <function rmse at 0x7fef95a4c320>
_____


### Test case

Here starts my (only) test case. The flow goes through cells defining module, mae and rmse, variable definitions, variable manipulation and _var_final_ being our target variable, being dependent of its previous cells.


In [4]:
def module(num): 
    if num >= 0:
        return int(num)
    else:
        return int(-1*num)

def mae(x, y):
    return np.mean(np.abs((x - y)))

def rmse(x, y):
    return np.sqrt(np.mean((x - y)**2))

def stress_func(m, n):
    temp = m * n
    x = 10
    y = 100
    
    temp = np.random.rand()*temp
    
    final = np.max([int(x+y), int(temp)])
    
    return final

In [5]:
var_a = 10

In [6]:
var_b = -100

In [7]:
x = 50
y = 30

In [8]:
rme_err = mae(var_a, var_b)

In [9]:
var_c = stress_func(x, y)

In [10]:
var_d = var_c * 234
var_c = var_d / 3

In [11]:
var_e = mae(var_c, var_b)
x = var_e + 10

In [12]:
#k = module(rmse_err)
var_final = now_variable('var_final', rmse(var_c, x))

Evaluation(id=139, checkpoint=32.972496052000004, code_component_id=734, activation_id=132, repr=110.0)


### New functions test

In [35]:
evals

evaluation(c7035a7f-34c5-4534-91cb-e88dc5367902, 139, 32.972496052000004, 734, 132, '110.0', 152).

In [36]:
evals = get_evaluation(139)
#code_comp_list = get_precedent([evals])

### Outputing the dependencies of a variable

print_precedents() trying to output a cleaner output from get_precedent() results

In [ ]:
def print_precedents(evals):
    code_componenents = get_precedent([evals])
    
    op_dict = {}
    for i in reversed(list(code_comp_list)):
        code_comp = i
        obj_eval = Evaluation((i.trial_id, i.id))
        obj_act = Activation((obj_eval.trial_id, obj_eval.activation_id))

        
        if  ((code_comp.type == 'call') and (code_comp.mode == 'r')) or \
            (code_comp.type == 'name' and code_comp.mode == 'w'): 
                op_dict[round(float(obj_eval.checkpoint), 6)] = [round(float(obj_act.start_checkpoint),6), \
                round(float(obj_eval.checkpoint), 6), code_comp.name, code_comp.type, code_comp.mode]  
    
    return op_dict

In [ ]:
print_precedents(evals)

In [ ]:
import pandas as pd

pd.DataFrame(my_dict, columns=['a'])